In [1]:
from strategy import DualReallocationStrategy
import dffc
from dffc import ReallocationBackTest as RBT
import json
import pandas as pd
import numpy as np

pd.set_option('future.no_silent_downcasting', True)

## Download & Update Data

In [2]:
codes = ['007467', '004253']
names=['HL', 'GD']
start_date = '2019-07-01'
end_date = '2025-07-01'

try:
    fund_data = dffc.FundData.load('hlvsgd_all.pkl')
except FileNotFoundError:
    fund_data = dffc.FundData.download(
        codes,
        names=names,
        # start=start_date,
        end=end_date
    )

    fund_data.save('hlvsgd_all.pkl')

fund_data = fund_data.update()
price_data_org = fund_data.get('unit_value') # .dropna()

In [3]:
fund_data.data['007467']

,unit_value,cumulative_value,daily_growth_rate,purchase_state,redemption_state,bonus_distribution
date,,,,,,
2017-05-02 00:00:00+08:00,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-03 00:00:00+08:00,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-04 00:00:00+08:00,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-05 00:00:00+08:00,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-08 00:00:00+08:00,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-11-27 00:00:00+08:00,1.6679,1.9579,0.0019,开放申购,开放赎回,
2025-11-28 00:00:00+08:00,1.6653,1.9553,-0.0016,开放申购,开放赎回,
2025-12-01 00:00:00+08:00,1.6760,1.9660,0.0064,开放申购,开放赎回,


In [4]:
estimate_p = dffc.FundEstimateProvider()

price_data = dffc.append_estimates_to_prices(
    price_data_org,
    codes,
    estimate_p,
    target_timezone=None,
)

Estimate data fetched for 007467 on 2025-12-03 15:00:00+08:00
Estimate data fetched for 004253 on 2025-12-03 22:28:00+08:00
No new estimate rows to append.


In [5]:
price_data

symbol,007467,004253
date,,
2017-05-02 00:00:00+08:00,NaN,1.0725
2017-05-03 00:00:00+08:00,NaN,1.0705
2017-05-04 00:00:00+08:00,NaN,1.0555
2017-05-05 00:00:00+08:00,NaN,1.0543
2017-05-08 00:00:00+08:00,NaN,1.0529
...,...,...
2025-11-27 00:00:00+08:00,1.6679,3.3387
2025-11-28 00:00:00+08:00,1.6653,3.3538
2025-12-01 00:00:00+08:00,1.6760,3.3898


## Load HW Parameter

In [5]:
with open('../hw_opt_results/demo_hw.json', 'r') as f:
    opt_res = json.load(f)

## Backtest Strategy

In [6]:
single_strategy = DualReallocationStrategy(
    prices = price_data,
    threshold=0.6,
    adjust_factor=0.2,
    default_weights = 0.5,
    up_weights = 0.2,
    down_weights= 0.8,
    tolerance=0.01,
    hw_params_list = opt_res
)

In [7]:
single_bt = RBT(
    strategy=single_strategy
)

single_bt.run(    
    start_date="2022-07-01",
    end_date=None,
    initial_cash=100_000,
    fees=0.0,
    trade_delay=0
)

2022-07-01 None
2022-07-01 00:00:00+08:00 None


100%|██████████| 1/1 [00:00<00:00,  1.26it/s]


In [8]:
single_bt.stats()

Start                           2022-07-01 00:00:00+08:00
End                             2025-12-03 00:00:00+08:00
Period                                  835 days 00:00:00
Start Value                                      100000.0
End Value                                   193244.505557
Total Return [%]                                93.244506
Benchmark Return [%]                            74.859158
Max Gross Exposure [%]                              100.0
Total Fees Paid                                       0.0
Max Drawdown [%]                                 7.351418
Max Drawdown Duration                    83 days 00:00:00
Total Trades                                          621
Total Closed Trades                                   619
Total Open Trades                                       2
Open Trade PnL                                6232.745684
Win Rate [%]                                    86.914378
Best Trade [%]                                  24.028208
Worst Trade [%

In [9]:
single_bt.plot()

In [12]:
single_strategy.hw_signals

,007467,004253
date,,
2022-07-01 00:00:00+08:00,0.287749,-0.806656
2022-07-04 00:00:00+08:00,0.116809,-0.629160
2022-07-05 00:00:00+08:00,0.198864,-0.645570
2022-07-06 00:00:00+08:00,-0.261364,-0.908228
2022-07-07 00:00:00+08:00,-0.280453,-0.952607
...,...,...
2025-11-26 00:00:00+08:00,-0.623529,-0.498565
2025-11-27 00:00:00+08:00,-0.579634,-0.076482
2025-11-28 00:00:00+08:00,-0.650131,0.195029


In [13]:
single_bt.plot(column = single_bt.pf.close.columns[1])

FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'cbae106a-e35b-496a-b8c8-47adac9e24a3',
              'x': array([Timestamp('2022-07-01 00:00:00+0800', tz='UTC+08:00'),
                          Timestamp('2022-07-04 00:00:00+0800', tz='UTC+08:00'),
                          Timestamp('2022-07-05 00:00:00+0800', tz='UTC+08:00'), ...,
                          Timestamp('2025-11-28 00:00:00+0800', tz='UTC+08:00'),
                          Timestamp('2025-12-01 00:00:00+0800', tz='UTC+08:00'),
                          Timestamp('2025-12-02 00:00:00+0800', tz='UTC+08:00')],
                         shape=(834,), dtype=object),
              'xaxis': 'x',
              'y': {'bdata': ('ppvEILBy9z/eAgmKH2P3P9v5fmq8dP' ... 'XgnBGl+j83iUFg5dD6P5huEoPAyvo/'),
                    'dtype': 'f8'},
              'yaxis': 'y'},
       

## Hyperparameter Optimization

In [6]:
strategy = DualReallocationStrategy(
    prices = price_data,
    threshold= np.arange(0.3, 2.0, 0.01),
    adjust_factor= np.arange(0.1, 1.1, 0.1),
    default_weights = 0.5,
    up_weights = np.arange(0.1, 0.5, 0.1),
    down_weights= np.arange(0.6, 1.0, 0.1),
    tolerance=0.01,
    hw_params_list = opt_res
)

In [7]:
bt = RBT(
    strategy=strategy,
    start_date="2022-07-01",
    end_date="2025-07-01",
    initial_cash=100_000,
    fees=0.,
    trade_delay=0
)

bt.run()

100%|██████████| 27200/27200 [11:50<00:00, 38.27it/s]


In [8]:
bt.stats() # selected=1

,,,,,,,benchmark_return,total_return,sharpe_ratio,max_drawdown
param_group,threshold,adjust_factor,default_weights,up_weights,down_weights,tolerance,,,,
0,0.30,0.1,0.5,0.1,0.6,0.01,0.540756,0.761648,2.018393,-0.052947
1,0.30,0.1,0.5,0.1,0.7,0.01,0.540756,0.744032,1.967199,-0.061328
2,0.30,0.1,0.5,0.1,0.8,0.01,0.540756,0.725808,1.891587,-0.069653
3,0.30,0.1,0.5,0.1,0.9,0.01,0.540756,0.707083,1.797507,-0.077921
4,0.30,0.1,0.5,0.2,0.6,0.01,0.540756,0.690909,1.876911,-0.057642
...,...,...,...,...,...,...,...,...,...,...
27195,1.99,1.0,0.5,0.3,0.9,0.01,0.540756,0.540756,1.508663,-0.065664
27196,1.99,1.0,0.5,0.4,0.6,0.01,0.540756,0.540756,1.508663,-0.065664
27197,1.99,1.0,0.5,0.4,0.7,0.01,0.540756,0.540756,1.508663,-0.065664


In [7]:
bt.plot(index_levels=['threshold', 'adjust_factor'], column_levels=['up_weights', 'down_weights'])

In [9]:
# Map top indices to their parameter sets without using an unhashable Index as a dict key
best_params = bt.get_best_param(metric='total_return', top_n=5)
best_params

,param_group,threshold,adjust_factor,default_weights,up_weights,down_weights,tolerance,total_return
0,7411,0.76,0.4,0.5,0.1,0.9,0.01,1.135884
1,7571,0.77,0.4,0.5,0.1,0.9,0.01,1.135884
2,7395,0.76,0.3,0.5,0.1,0.9,0.01,1.129134
3,7555,0.77,0.3,0.5,0.1,0.9,0.01,1.129134
4,7427,0.76,0.5,0.5,0.1,0.9,0.01,1.127818


In [10]:
best_params= bt.get_weighted_best_params(
    metric_weight = {
        'total_return': 0.45,
        'sharpe_ratio': 0.45,
        'max_drawdown': 0.1
    },
    top_n=10
)

In [11]:
best_params

,param_group,threshold,adjust_factor,default_weights,up_weights,down_weights,tolerance,weighted_score,total_return,sharpe_ratio,max_drawdown
0,7394,0.76,0.3,0.5,0.1,0.8,0.01,0.943068,1.090581,2.378682,-0.070904
1,7554,0.77,0.3,0.5,0.1,0.8,0.01,0.943068,1.090581,2.378682,-0.070904
2,7553,0.77,0.3,0.5,0.1,0.7,0.01,0.942933,1.054875,2.410353,-0.062222
3,7393,0.76,0.3,0.5,0.1,0.7,0.01,0.942933,1.054875,2.410353,-0.062222
4,7570,0.77,0.4,0.5,0.1,0.8,0.01,0.940524,1.098267,2.360741,-0.072587
5,7410,0.76,0.4,0.5,0.1,0.8,0.01,0.940524,1.098267,2.360741,-0.072587
6,7555,0.77,0.3,0.5,0.1,0.9,0.01,0.938446,1.129134,2.322774,-0.079690
7,7395,0.76,0.3,0.5,0.1,0.9,0.01,0.938446,1.129134,2.322774,-0.079690
8,7569,0.77,0.4,0.5,0.1,0.7,0.01,0.935924,1.055579,2.387905,-0.063761
9,7409,0.76,0.4,0.5,0.1,0.7,0.01,0.935924,1.055579,2.387905,-0.063761


## Best Param Strategy

In [12]:
best_strategy = DualReallocationStrategy(
    prices = price_data,
    threshold = best_params['threshold'][0],
    adjust_factor = best_params['adjust_factor'][0],
    default_weights = 0.,
    up_weights = best_params['up_weights'][0],
    down_weights = best_params['down_weights'][0],
    tolerance=0.01,
    hw_params_list = opt_res
)

In [16]:
best_bt = RBT(
    strategy=best_strategy,
    start_date="2022-07-01",
    # end_date="2025-07-02",
    initial_cash=20_000,
    fees=0.0,
    trade_delay=0
)

best_bt.run()

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  6.54it/s]


In [17]:
best_strategy.hw_signals

,007467,004253
date,,
2024-07-17 00:00:00+08:00,-0.556667,0.946667
2024-07-18 00:00:00+08:00,-0.384359,0.920133
2024-07-19 00:00:00+08:00,-0.607321,-0.670549
2024-07-22 00:00:00+08:00,-0.813953,-0.239203
2024-07-23 00:00:00+08:00,-0.850498,-0.677741
...,...,...
2025-10-23 00:00:00+08:00,0.816733,-0.399734
2025-10-24 00:00:00+08:00,0.493369,-0.864721
2025-10-27 00:00:00+08:00,0.506631,-0.949602


In [18]:
best_bt.pf.orders.records_readable

,Order Id,Column,Timestamp,Size,Price,Fees,Side
0,0,"(0, 0.92, 0.8, 0.5, 0.1, 0.6, 0.01, 004253)",2024-07-17 00:00:00+08:00,4811.16,2.0785,0.0,Buy
1,1,"(0, 0.92, 0.8, 0.5, 0.1, 0.6, 0.01, 007467)",2024-07-17 00:00:00+08:00,6259.78,1.5975,0.0,Buy
2,2,"(0, 0.92, 0.8, 0.5, 0.1, 0.6, 0.01, 004253)",2024-07-18 00:00:00+08:00,761.85,2.0788,0.0,Sell
3,3,"(0, 0.92, 0.8, 0.5, 0.1, 0.6, 0.01, 007467)",2024-07-18 00:00:00+08:00,987.36,1.6040,0.0,Buy
4,4,"(0, 0.92, 0.8, 0.5, 0.1, 0.6, 0.01, 004253)",2024-07-19 00:00:00+08:00,110.94,2.0243,0.0,Sell
5,5,"(0, 0.92, 0.8, 0.5, 0.1, 0.6, 0.01, 007467)",2024-07-19 00:00:00+08:00,141.08,1.5919,0.0,Buy
6,6,"(0, 0.92, 0.8, 0.5, 0.1, 0.6, 0.01, 007467)",2024-10-09 00:00:00+08:00,4871.34,1.5855,0.0,Sell
7,7,"(0, 0.92, 0.8, 0.5, 0.1, 0.6, 0.01, 004253)",2024-10-09 00:00:00+08:00,3656.62,2.1122,0.0,Buy
8,8,"(0, 0.92, 0.8, 0.5, 0.1, 0.6, 0.01, 007467)",2024-10-10 00:00:00+08:00,1044.61,1.6480,0.0,Sell
9,9,"(0, 0.92, 0.8, 0.5, 0.1, 0.6, 0.01, 004253)",2024-10-10 00:00:00+08:00,812.19,2.1196,0.0,Buy


In [17]:
best_bt.stats()

Start                           2022-07-01 00:00:00+08:00
End                             2025-11-28 00:00:00+08:00
Period                                  832 days 00:00:00
Start Value                                       20000.0
End Value                                    46003.762378
Total Return [%]                               130.018812
Benchmark Return [%]                            74.551814
Max Gross Exposure [%]                              100.0
Total Fees Paid                                       0.0
Max Drawdown [%]                                 7.090415
Max Drawdown Duration                    53 days 00:00:00
Total Trades                                          431
Total Closed Trades                                   429
Total Open Trades                                       2
Open Trade PnL                                2221.843036
Win Rate [%]                                    84.615385
Best Trade [%]                                  24.047225
Worst Trade [%

In [18]:
best_bt.plot()

In [18]:
best_bt.pf.orders.records_readable


,Order Id,Column,Timestamp,Size,Price,Fees,Side
0,0,"(0, 0.83, 0.2, 0.0, 0.1, 0.6, 0.01, 004253)",2025-05-28 00:00:00+08:00,7295.80,2.7413,0.0,Buy
1,1,"(0, 0.83, 0.2, 0.0, 0.1, 0.6, 0.01, 004253)",2025-05-29 00:00:00+08:00,145.90,2.7174,0.0,Sell
2,2,"(0, 0.83, 0.2, 0.0, 0.1, 0.6, 0.01, 007467)",2025-05-29 00:00:00+08:00,242.51,1.6349,0.0,Buy
3,3,"(0, 0.83, 0.2, 0.0, 0.1, 0.6, 0.01, 004253)",2025-05-30 00:00:00+08:00,117.36,2.7379,0.0,Sell
4,4,"(0, 0.83, 0.2, 0.0, 0.1, 0.6, 0.01, 007467)",2025-05-30 00:00:00+08:00,195.94,1.6399,0.0,Buy
...,...,...,...,...,...,...,...
98,98,"(0, 0.83, 0.2, 0.0, 0.1, 0.6, 0.01, 004253)",2025-10-27 00:00:00+08:00,435.49,3.2970,0.0,Buy
99,99,"(0, 0.83, 0.2, 0.0, 0.1, 0.6, 0.01, 007467)",2025-10-28 00:00:00+08:00,766.79,1.6626,0.0,Sell
100,100,"(0, 0.83, 0.2, 0.0, 0.1, 0.6, 0.01, 004253)",2025-10-28 00:00:00+08:00,401.36,3.1764,0.0,Buy
101,101,"(0, 0.83, 0.2, 0.0, 0.1, 0.6, 0.01, 007467)",2025-10-29 00:00:00+08:00,488.22,1.6492,0.0,Sell


In [19]:
delta_hdp = - best_strategy.hw_signals.iloc[:, 0] + best_strategy.hw_signals.iloc[:, 1]

In [20]:
best_strategy.hw_signals

,007467,004253
date,,
2025-05-28 00:00:00+08:00,0.335227,-0.446023
2025-05-29 00:00:00+08:00,0.232955,-0.877841
2025-05-30 00:00:00+08:00,0.293617,-0.273759
2025-06-03 00:00:00+08:00,0.690780,0.571631
2025-06-04 00:00:00+08:00,0.742210,0.399433
...,...,...
2025-10-23 00:00:00+08:00,0.819389,-0.776892
2025-10-24 00:00:00+08:00,0.498674,-0.928382
2025-10-27 00:00:00+08:00,0.509284,-0.954907


In [21]:
delta_hdp

date
2025-05-28 00:00:00+08:00   -0.781250
2025-05-29 00:00:00+08:00   -1.110795
2025-05-30 00:00:00+08:00   -0.567376
2025-06-03 00:00:00+08:00   -0.119149
2025-06-04 00:00:00+08:00   -0.342776
                               ...   
2025-10-23 00:00:00+08:00   -1.596282
2025-10-24 00:00:00+08:00   -1.427056
2025-10-27 00:00:00+08:00   -1.464191
2025-10-28 00:00:00+08:00   -1.480795
2025-10-29 00:00:00+08:00   -0.966887
Length: 104, dtype: float64